### Imports

In [3]:
from typing import Optional, Tuple

import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input

In [4]:
np.set_printoptions(linewidth=np.inf)

### Data generation

In [5]:
n_sample = 120
data = pd.DataFrame(
    columns=["v1", "v2", "t"],
    index=pd.date_range(start="2023-01-01", freq="D", periods=n_sample),
)
data["v1"] = np.arange(1, n_sample + 1)
data["v2"] = data["v1"] + 0.1
data["t"] = data["v1"] + 0.01

print(f"Data shape {data.shape}")

data.head()

Data shape (120, 3)


,v1,v2,t
2023-01-01,1,1.1,1.01
2023-01-02,2,2.1,2.01
2023-01-03,3,3.1,3.01
2023-01-04,4,4.1,4.01
2023-01-05,5,5.1,5.01


### Test Train Split

In [6]:
train_data = data.head(100)
test_data = data.tail(20)
train_x, train_y = train_data[["v1", "v2"]].copy(deep=True), train_data[["t"]].copy(
    deep=True
)
test_x, test_y = test_data[["v1", "v2"]].copy(deep=True), test_data[["t"]].copy(
    deep=True
)

print(f"train_x shape {train_x.shape}, train_y shape {train_y.shape}")
print(f"test_x shape  {test_x.shape},  test_y  shape {test_y.shape}")

train_x shape (100, 2), train_y shape (100, 1)
test_x shape  (20, 2),  test_y  shape (20, 1)


### SIMPLE LSTM DENSE MODEL

In [7]:
def lstm_model(shape: Tuple[int, int], n_unit: int = 64):
    model = Sequential(
        [
            LSTM(units=n_unit, return_sequences=True, input_shape=shape, dropout=0.2),
            LSTM(units=n_unit, return_sequences=True, dropout=0.2),
            LSTM(units=n_unit, dropout=0.2),
            Dense(units=1),
        ],
        name="LSTM_3_DENSE",
    )
    model.compile(optimizer="adam", loss="mse", metrics="mae")
    return model

### Forecaster Wrapper

In [13]:
class LstmForecaster:
    def __init__(self, window_len: int = 10, epoch: int = 25, batch_size: int = 64):
        self.window_len = window_len
        self.epoch = epoch
        self.batch_size = batch_size
        self.data_x_max, self.data_x_min = None, None
        self.data_y_max, self.data_y_min = None, None
        self.initial_state = None
        self.forecaster = None

    def minmax_scale_x(self, data_x: pd.DataFrame) -> pd.DataFrame:
        return (data_x - self.data_x_min) / (self.data_x_max - self.data_x_min)

    def minmax_scale_y(self, data_y: pd.Series) -> pd.Series:
        return (data_y - self.data_y_min) / (self.data_y_max - self.data_y_min)

    def inv_minmax_scale_y(self, data_y: pd.Series) -> pd.Series:
        return data_y * (self.data_y_max - self.data_y_min) + self.data_y_min

    def preprocess_data(
        self, data_x: pd.DataFrame, data_y: pd.Series
    ) -> Tuple[np.ndarray, np.ndarray]:
        x_data, y_data = [], []

        data_size = data_x.shape[0]

        self.data_x_max, self.data_x_min = data_x.max(), data_x.min()
        self.data_y_max, self.data_y_min = data_y.max().item(), data_y.min().item()

        x_scaled = self.minmax_scale_x(data_x)
        y_scaled = self.minmax_scale_y(data_y)

        # x_scaled = data_x
        # y_scaled = data_y

        xy_scaled = pd.concat([x_scaled, y_scaled], axis=1)

        self.initial_state = xy_scaled.tail(self.window_len).to_numpy()[np.newaxis, :]

        for i in range(self.window_len, data_size):
            x_data.append(xy_scaled.iloc[i - self.window_len : i, :])
            y_data.append(y_scaled.iloc[i, :])
        return np.array(x_data, dtype=np.float64), np.array(y_data, dtype=np.float64)

    def fit(self, X: pd.DataFrame, y: pd.Series):
        x_data, y_data = self.preprocess_data(X, y)
        self.forecaster = lstm_model(x_data.shape[1:])

        self.forecaster.fit(
            x_data, y_data, epochs=self.epoch, batch_size=self.batch_size, shuffle=False
        )
        return self

    def predict(self, forecast_horizon: int, X: pd.DataFrame) -> np.ndarray:
        y_predict = []
        x_scaled = self.minmax_scale_x(X)
        model_input = self.initial_state
        for i in range(forecast_horizon):
            predict_t = self.forecaster.predict(model_input, verbose=0)
            y_predict.append(predict_t.flatten().item())
            xhog = x_scaled.iloc[i, :].to_numpy()
            new_test = np.hstack([xhog, predict_t.flatten()])
            model_input_2d = model_input.reshape(self.window_len, -1)
            model_input_2d = np.vstack([model_input_2d, new_test])[-self.window_len :]
            model_input = model_input_2d[np.newaxis, :]
        y_original_scale = self.inv_minmax_scale_y(np.array(y_predict))
        return y_original_scale

### Training And Forecasting

In [14]:
fcaster = LstmForecaster(window_len=4, epoch=25)
x_, y_ = fcaster.preprocess_data(train_x, train_y)

print(x_.shape, y_.shape, fcaster.initial_state.shape)
print(x_)
print(y_)
print(fcaster.initial_state)

(96, 4, 3) (96, 1) (1, 4, 3)
[[[0.         0.         0.        ]
  [0.01010101 0.01010101 0.01010101]
  [0.02020202 0.02020202 0.02020202]
  [0.03030303 0.03030303 0.03030303]]

 [[0.01010101 0.01010101 0.01010101]
  [0.02020202 0.02020202 0.02020202]
  [0.03030303 0.03030303 0.03030303]
  [0.04040404 0.04040404 0.04040404]]

 [[0.02020202 0.02020202 0.02020202]
  [0.03030303 0.03030303 0.03030303]
  [0.04040404 0.04040404 0.04040404]
  [0.05050505 0.05050505 0.05050505]]

 ...

 [[0.93939394 0.93939394 0.93939394]
  [0.94949495 0.94949495 0.94949495]
  [0.95959596 0.95959596 0.95959596]
  [0.96969697 0.96969697 0.96969697]]

 [[0.94949495 0.94949495 0.94949495]
  [0.95959596 0.95959596 0.95959596]
  [0.96969697 0.96969697 0.96969697]
  [0.97979798 0.97979798 0.97979798]]

 [[0.95959596 0.95959596 0.95959596]
  [0.96969697 0.96969697 0.96969697]
  [0.97979798 0.97979798 0.97979798]
  [0.98989899 0.98989899 0.98989899]]]
[[0.04040404]
 [0.05050505]
 [0.06060606]
 [0.07070707]
 [0.08080

In [15]:
fcaster = LstmForecaster(window_len=10, epoch=25)
fcaster.fit(train_x, train_y)

y_pred = fcaster.predict(20, test_x)

Epoch 1/25
2/2 [==============================] - 6s 26ms/step - loss: 0.3155 - mae: 0.5138
Epoch 2/25
2/2 [==============================] - 0s 10ms/step - loss: 0.1979 - mae: 0.4027
Epoch 3/25
2/2 [==============================] - 0s 22ms/step - loss: 0.0876 - mae: 0.2641
Epoch 4/25
2/2 [==============================] - 0s 10ms/step - loss: 0.0109 - mae: 0.0836
Epoch 5/25
2/2 [==============================] - 0s 22ms/step - loss: 0.0344 - mae: 0.1585
Epoch 6/25
2/2 [==============================] - 0s 9ms/step - loss: 0.0624 - mae: 0.2369
Epoch 7/25
2/2 [==============================] - 0s 21ms/step - loss: 0.0228 - mae: 0.1410
Epoch 8/25
2/2 [==============================] - 0s 11ms/step - loss: 0.0092 - mae: 0.0831
Epoch 9/25
2/2 [==============================] - 0s 26ms/step - loss: 0.0150 - mae: 0.0922
Epoch 10/25
2/2 [==============================] - 0s 20ms/step - loss: 0.0190 - mae: 0.1060
Epoch 11/25
2/2 [==============================] - 0s 10ms/step - loss: 0.0152 -

### 20 Point forecasted results

In [ ]:
y_pred

In [ ]:
np.hstack([test_x.to_numpy(), y_pred.reshape(-1, 1)])